In [1]:
import pandas as pd

pathToGungorVict = '..//..//DS7004//u1720146_DS7004_courseworkCodeAndData//preparationWorks//fromDS7003_Gungor2018VictorianAuthorAttribution_NGram//Gungor_2018_VictorianAuthorAttribution_data-train.csv'
gungorVictRow = pd.read_csv(pathToGungorVict, encoding = 'ISO-8859-1')
#26 Jane Austen 4441 rows/ 45 Thomas Hardy 2312 rows/ 8 Charles Dickens 6914 rows

In [2]:
 jausten = gungorVictRow.loc[gungorVictRow['author'] == 26]

In [3]:
jausten.shape, type(jausten)

((4441, 2), pandas.core.frame.DataFrame)

In [4]:
thardy = gungorVictRow.loc[gungorVictRow['author'] == 45]

In [5]:
thardy.shape, type(jausten)

((2312, 2), pandas.core.frame.DataFrame)

In [6]:
cdickens = gungorVictRow.loc[gungorVictRow['author'] == 8] #SCharles Dickens

In [7]:
cdickens.shape, type(cdickens)

((6914, 2), pandas.core.frame.DataFrame)

In [8]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [9]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
#nltk.download() no need to use this line after using it once(?)   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [10]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [11]:
def parsing_sentence_set(text_df):
    sentences = []  # Initialize an empty list of sentences

    print("Parsing sentences from training set")
    for review in text_df["text"]:
        sentences += review_to_sentences(review, tokenizer)
    return sentences

In [12]:
jausten_sentences = parsing_sentence_set(jausten)

Parsing sentences from training set


In [18]:
thardy_sentences = parsing_sentence_set(thardy)

Parsing sentences from training set


In [19]:
cdickens_sentences = parsing_sentence_set(cdickens)

Parsing sentences from training set


In [20]:
# Check how many paragraphs of jane austen's writing we have in total
print(len(jausten_sentences))

4441


In [16]:
# Check how many paragraphs of thomas hardy's writing we have in total
print(len(thardy_sentences))

2312


In [21]:
# Check how many paragraphs of charles dickens's writing we have in total
print(len(cdickens_sentences))

6914


In [22]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
#num_features = 300    # Word vector dimensionality                      
#min_word_count = 40   # Minimum word count                        
#num_workers = 4       # Number of threads to run in parallel
#context = 10          # Context window size                                                                                    
#downsampling = 1e-3   # Downsample setting for frequent words

num_features = 300    # Word vector dimensionality                      
min_word_count = 5    # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
epochs=15             #number of epochs

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
def form_model_from_sentences(sentences):
    print("Training model...")
    model = word2vec.Word2Vec(sentences, workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = context, sample = downsampling, iter = epochs)

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)
    return model

In [23]:
jausten_model = form_model_from_sentences(jausten_sentences)

2020-08-21 22:30:52,057 : INFO : collecting all words and their counts
2020-08-21 22:30:52,058 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2020-08-21 22:30:52,707 : INFO : collected 8998 word types from a corpus of 4409095 raw words and 4441 sentences
2020-08-21 22:30:52,708 : INFO : Loading a fresh vocabulary
2020-08-21 22:30:52,721 : INFO : effective_min_count=5 retains 8199 unique words (91% of original 8998, drops 799)
2020-08-21 22:30:52,721 : INFO : effective_min_count=5 leaves 4407100 word corpus (99% of original 4409095, drops 1995)
2020-08-21 22:30:52,742 : INFO : deleting the raw counts dictionary of 8998 items
2020-08-21 22:30:52,743 : INFO : sample=0.001 downsamples 62 most-common words
2020-08-21 22:30:52,744 : INFO : downsampling leaves estimated 3098040 word corpus (70.3% of prior 4407100)
2020-08-21 22:30:52,764 : INFO : estimated required memory for 8199 words and 300 dimensions: 23777100 bytes
2020-08-21 22:30:52,764 : INFO : resetting layer weights
2020-08-21 22:30:54,101 : INFO : training model with 4 workers on 8199 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2020-08

2020-08-21 22:31:21,606 : INFO : EPOCH 11 - PROGRESS: at 33.33% examples, 1017761 words/s, in_qsize 7, out_qsize 0
2020-08-21 22:31:22,612 : INFO : EPOCH 11 - PROGRESS: at 67.55% examples, 1035610 words/s, in_qsize 7, out_qsize 0
2020-08-21 22:31:23,546 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-08-21 22:31:23,549 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-21 22:31:23,553 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-21 22:31:23,562 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-21 22:31:23,563 : INFO : EPOCH - 11 : training on 4409095 raw words (3097330 effective words) took 3.0s, 1042660 effective words/s
2020-08-21 22:31:24,569 : INFO : EPOCH 12 - PROGRESS: at 33.55% examples, 1033318 words/s, in_qsize 7, out_qsize 0
2020-08-21 22:31:25,573 : INFO : EPOCH 12 - PROGRESS: at 68.68% examples, 1059086 words/s, in_qsize 7, out_qsize 0
2020-08-21 22:31:26,461 : INFO : work

In [24]:
jausten_model.doesnt_match("man woman child kitchen".split())

<ipython-input-24-d24306086988>:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  jausten_model.doesnt_match("man woman child kitchen".split())
C:\Users\Eric\anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

In [25]:
jausten_model.doesnt_match("paris bath london kitchen".split())

<ipython-input-25-a3bec1ffe496>:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  jausten_model.doesnt_match("paris bath london kitchen".split())


'paris'

In [26]:
jausten_model.most_similar(positive=['king', 'woman'], negative=['man'])

<ipython-input-26-f51a07e79a64>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  jausten_model.most_similar(positive=['king', 'woman'], negative=['man'])


[('bids', 0.27594566345214844),
 ('marble', 0.2721642553806305),
 ('flown', 0.27040964365005493),
 ('bathed', 0.26802968978881836),
 ('shakespeare', 0.26359665393829346),
 ('queen', 0.2634279131889343),
 ('published', 0.2630871832370758),
 ('vo', 0.2612764835357666),
 ('lofty', 0.2598876953125),
 ('songs', 0.25884634256362915)]

In [27]:
jausten_model.most_similar("man")

<ipython-input-27-8cbd6bff1b7d>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  jausten_model.most_similar("man")


[('woman', 0.5145279169082642),
 ('fellow', 0.44151949882507324),
 ('associated', 0.3969119191169739),
 ('person', 0.3726927638053894),
 ('people', 0.36004072427749634),
 ('men', 0.351967453956604),
 ('scholar', 0.3365447521209717),
 ('girl', 0.3285280168056488),
 ('blunt', 0.3281540870666504),
 ('proprietor', 0.32804304361343384)]

In [28]:
jausten_model.most_similar("woman")

<ipython-input-28-4efcc6cfe590>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  jausten_model.most_similar("woman")


[('man', 0.5145279169082642),
 ('girl', 0.4219675064086914),
 ('women', 0.38523733615875244),
 ('person', 0.35929715633392334),
 ('inexperienced', 0.3492203950881958),
 ('associated', 0.32224810123443604),
 ('creature', 0.3205755949020386),
 ('amiable', 0.3195722699165344),
 ('figure', 0.312466025352478),
 ('heroine', 0.2851046323776245)]

In [29]:
thardy_model = form_model_from_sentences(thardy_sentences)

2020-08-21 22:31:35,436 : INFO : collecting all words and their counts
2020-08-21 22:31:35,437 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2020-08-21 22:31:36,304 : INFO : collected 9952 word types from a corpus of 2292403 raw words and 2312 sentences
2020-08-21 22:31:36,305 : INFO : Loading a fresh vocabulary
2020-08-21 22:31:36,337 : INFO : effective_min_count=5 retains 9517 unique words (95% of original 9952, drops 435)
2020-08-21 22:31:36,338 : INFO : effective_min_count=5 leaves 2291116 word corpus (99% of original 2292403, drops 1287)
2020-08-21 22:31:36,402 : INFO : deleting the raw counts dictionary of 9952 items
2020-08-21 22:31:36,403 : INFO : sample=0.001 downsamples 52 most-common words
2020-08-21 22:31:36,404 : INFO : downsampling leaves estimated 1627151 word corpus (71.0% of prior 2291116)
2020-08-21 22:31:36,459 : INFO : estimated required memory for 9517 words and 300 dimensions: 27599300 bytes
2020-08-21 22:31:36,460 : INFO : resetting layer weights
2020-08-21 22:31:38,770 : INFO : training model with 4 workers on 9517 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2020-08

2020-08-21 22:32:00,341 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-21 22:32:00,346 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-21 22:32:00,351 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-21 22:32:00,352 : INFO : EPOCH - 12 : training on 2292403 raw words (1627363 effective words) took 2.7s, 611592 effective words/s
2020-08-21 22:32:01,355 : INFO : EPOCH 13 - PROGRESS: at 67.04% examples, 1088976 words/s, in_qsize 7, out_qsize 0
2020-08-21 22:32:01,839 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-08-21 22:32:01,847 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-21 22:32:01,851 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-21 22:32:01,856 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-21 22:32:01,856 : INFO : EPOCH - 13 : training on 2292403 raw words (1627790 effective words) took 1.5

In [30]:
thardy_model.doesnt_match("man woman child kitchen".split())

<ipython-input-30-e5a623de7caf>:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  thardy_model.doesnt_match("man woman child kitchen".split())


'kitchen'

In [31]:
thardy_model.doesnt_match("paris bath london kitchen".split())

<ipython-input-31-9d9b13329596>:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  thardy_model.doesnt_match("paris bath london kitchen".split())


'kitchen'

In [32]:
thardy_model.most_similar(positive=['king', 'woman'], negative=['man'])

<ipython-input-32-7108fa9f8ec2>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  thardy_model.most_similar(positive=['king', 'woman'], negative=['man'])


[('st', 0.39107972383499146),
 ('daughter', 0.36646023392677307),
 ('court', 0.36549362540245056),
 ('princess', 0.3591286838054657),
 ('feast', 0.35720276832580566),
 ('castle', 0.3356744647026062),
 ('graciously', 0.33558088541030884),
 ('situated', 0.3263382315635681),
 ('edward', 0.32416361570358276),
 ('rev', 0.32398712635040283)]

In [33]:
thardy_model.most_similar("man")

<ipython-input-33-0710a1c3ca7f>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  thardy_model.most_similar("man")


[('woman', 0.5533199906349182),
 ('gentleman', 0.5480035543441772),
 ('fellow', 0.5450979471206665),
 ('person', 0.49691176414489746),
 ('nobleman', 0.4953759014606476),
 ('traveller', 0.44964003562927246),
 ('soldier', 0.44952404499053955),
 ('creature', 0.4448680579662323),
 ('foreigner', 0.44305747747421265),
 ('girl', 0.4378548562526703)]

In [34]:
thardy_model.most_similar("woman")

<ipython-input-34-375cdea8604a>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  thardy_model.most_similar("woman")


[('girl', 0.5574241876602173),
 ('man', 0.5533199906349182),
 ('creature', 0.49580860137939453),
 ('lover', 0.45769014954566956),
 ('lady', 0.4545402228832245),
 ('person', 0.4490666687488556),
 ('criminal', 0.40846365690231323),
 ('foreigner', 0.3867684006690979),
 ('maid', 0.3711322247982025),
 ('gentleman', 0.353089302778244)]

In [36]:
cdickens_model = form_model_from_sentences(cdickens_sentences)

2020-08-21 22:32:54,071 : INFO : collecting all words and their counts
2020-08-21 22:32:54,072 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2020-08-21 22:32:55,358 : INFO : collected 9970 word types from a corpus of 6855141 raw words and 6914 sentences
2020-08-21 22:32:55,358 : INFO : Loading a fresh vocabulary
2020-08-21 22:32:55,374 : INFO : effective_min_count=5 retains 9931 unique words (99% of original 9970, drops 39)
2020-08-21 22:32:55,374 : INFO : effective_min_count=5 leaves 6855037 word corpus (99% of original 6855141, drops 104)
2020-08-21 22:32:55,403 : INFO : deleting the raw counts dictionary of 9970 items
2020-08-21 22:32:55,404 : INFO : sample=0.001 downsamples 57 most-common words
2020-08-21 22:32:55,404 : INFO : downsampling leaves estimated 4840154 word corpus (70.6% of prior 6855037)
2020-08-21 22:32:55,431 : INFO : estimated required memory for 9931 words and 300 dimensions: 28799900 bytes
2020-08-21 22:32:55,431 : INFO : resetting layer weights
2020-08-21 22:32:57,076 : INFO : training model with 4 workers on 9931 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2020-08-2

2020-08-21 22:33:33,945 : INFO : EPOCH 8 - PROGRESS: at 21.84% examples, 1034289 words/s, in_qsize 7, out_qsize 0
2020-08-21 22:33:34,971 : INFO : EPOCH 8 - PROGRESS: at 35.29% examples, 832254 words/s, in_qsize 7, out_qsize 0
2020-08-21 22:33:35,977 : INFO : EPOCH 8 - PROGRESS: at 47.58% examples, 753119 words/s, in_qsize 8, out_qsize 0
2020-08-21 22:33:36,988 : INFO : EPOCH 8 - PROGRESS: at 59.44% examples, 708639 words/s, in_qsize 7, out_qsize 0
2020-08-21 22:33:37,989 : INFO : EPOCH 8 - PROGRESS: at 79.11% examples, 757007 words/s, in_qsize 7, out_qsize 0
2020-08-21 22:33:38,878 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-08-21 22:33:38,887 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-21 22:33:38,891 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-21 22:33:38,892 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-21 22:33:38,892 : INFO : EPOCH - 8 : training on 6855141 raw w

2020-08-21 22:34:18,113 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-21 22:34:18,114 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-21 22:34:18,115 : INFO : EPOCH - 15 : training on 6855141 raw words (4840008 effective words) took 5.8s, 827499 effective words/s
2020-08-21 22:34:18,116 : INFO : training on a 102827115 raw words (72601926 effective words) took 81.0s, 895882 effective words/s
2020-08-21 22:34:18,116 : INFO : precomputing L2-norms of word weight vectors


In [37]:
cdickens_model.doesnt_match("man woman child kitchen".split())

<ipython-input-37-4d4030d416e9>:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  cdickens_model.doesnt_match("man woman child kitchen".split())


'kitchen'

In [38]:
cdickens_model.doesnt_match("paris bath london kitchen".split())

<ipython-input-38-25d03882a25b>:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  cdickens_model.doesnt_match("paris bath london kitchen".split())


'bath'

In [39]:
cdickens_model.most_similar(positive=['king', 'woman'], negative=['man'])

<ipython-input-39-08444b34ff55>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  cdickens_model.most_similar(positive=['king', 'woman'], negative=['man'])


[('queen', 0.5306344032287598),
 ('princess', 0.3572227954864502),
 ('earl', 0.3498882055282593),
 ('france', 0.34229129552841187),
 ('england', 0.32383811473846436),
 ('duke', 0.31832751631736755),
 ('she', 0.31116506457328796),
 ('prince', 0.3074086904525757),
 ('clergy', 0.3068374991416931),
 ('castle', 0.30129289627075195)]

In [40]:
cdickens_model.most_similar("man")

<ipython-input-40-87b35789ec91>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  cdickens_model.most_similar("man")


[('gentleman', 0.5247000455856323),
 ('woman', 0.49636879563331604),
 ('chap', 0.4661363959312439),
 ('fellow', 0.44961440563201904),
 ('madman', 0.4260881245136261),
 ('person', 0.4052414000034332),
 ('liar', 0.3811708688735962),
 ('dog', 0.3796888589859009),
 ('jew', 0.37754103541374207),
 ('scoundrel', 0.3694858253002167)]

In [41]:
cdickens_model.most_similar("woman")

<ipython-input-41-dfbe8c43d0ad>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  cdickens_model.most_similar("woman")


[('lady', 0.5765020847320557),
 ('man', 0.49636879563331604),
 ('girl', 0.44128143787384033),
 ('creature', 0.4041684865951538),
 ('chap', 0.38590413331985474),
 ('lover', 0.38030850887298584),
 ('person', 0.3802489638328552),
 ('mistress', 0.3741688132286072),
 ('she', 0.3709544539451599),
 ('un', 0.3691060543060303)]